In [ ]:
%aiida
from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)

In [ ]:
from aiida.orm.data.structure import StructureData
from aiida.orm.data.parameter import ParameterData
from aiida.orm.data.array import ArrayData
from aiida.orm.data.base import Int, Str, Float, Bool
from aiida.orm.data.remote import RemoteData
from aiida.work import workfunction
from aiida.work.process import WorkCalculation
from aiida.work.run import submit
from aiida_cp2k.calculations import Cp2kCalculation

from aiida.orm import Code, Computer
from aiida.orm.querybuilder import QueryBuilder

import ase
import ase.io
import numpy as np
import nglview
from copy import deepcopy
from pprint import pprint

import ipywidgets as ipw
from IPython.display import display, clear_output, HTML

from apps.scanning_probe.stm.stm_workchain import STMWorkChain


# Select structure

In [ ]:
from apps.surfaces.structure_browser import StructureBrowser

atoms = None

def on_struct_change(c):
    global atoms
    structure = struct_browser.results.value
    if structure:
        atoms = structure.get_ase()
        atoms.pbc = [1, 1, 1]
        
        unobserve_cell_text()
        cell_x_text.value = atoms.cell[0,0]
        cell_y_text.value = atoms.cell[1,1]
        cell_z_text.value = atoms.cell[2,2]
        observe_cell_text()
    
    update_view(atoms)

def on_cell_change(c):
    global atoms
    atoms.cell = np.diag(np.array([
        cell_x_text.value,
        cell_y_text.value,
        cell_z_text.value
    ]))
    atoms.center()
    update_view(atoms)
    
def observe_cell_text():
    cell_x_text.observe(on_cell_change, names='value')
    cell_y_text.observe(on_cell_change, names='value')
    cell_z_text.observe(on_cell_change, names='value')
    
def unobserve_cell_text():
    cell_x_text.unobserve(on_cell_change, names='value')
    cell_y_text.unobserve(on_cell_change, names='value')
    cell_z_text.unobserve(on_cell_change, names='value')
    
struct_browser = StructureBrowser()
struct_browser.results.observe(on_struct_change, names='value')
viewer = nglview.NGLWidget()
clear_output()

cell_x_text = ipw.FloatText(description='cell x', value=0.0)
cell_y_text = ipw.FloatText(description='cell y', value=0.0)
cell_z_text = ipw.FloatText(description='cell z', value=0.0)

observe_cell_text()

display(ipw.VBox([struct_browser, viewer, cell_x_text, cell_y_text, cell_z_text]))

def update_view(atoms):
    
    # remove old components
    if hasattr(viewer, "component_0"):
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_unitcell()
        cid = viewer.component_0.id
        viewer.remove_component(cid)
    
    if atoms:
        # add new component
        viewer.add_component(nglview.ASEStructure(atoms)) # adds ball+stick
        viewer.add_unitcell()
        viewer.center()
        
        # Orient camera to look from positive z
        cell_z = atoms.cell[2, 2]
        com = atoms.get_center_of_mass()
        def_orientation = viewer._camera_orientation
        top_z_orientation = [1.0, 0.0, 0.0, 0,
                             0.0, 1.0, 0.0, 0,
                             0.0, 0.0, -np.max([cell_z, 30.0]) , 0,
                             -com[0], -com[1], -com[2], 1]
        viewer._set_camera_orientation(top_z_orientation)
        #viewer.camera = 'orthographic'

# Select computer and codes

In [ ]:
qb = QueryBuilder()
qb.append(Computer, filters={'enabled': True}, project='name')
computer_names = [comp[0] for comp in qb.all()]

style = {'description_width': '120px'}
layout = {'width': '70%'}
drop_computer = ipw.Dropdown(description="Computer",
                             options=computer_names)

def comp_plugin_codes(computer_name, plugin_name):
    qb = QueryBuilder()
    qb.append(Computer, filters={'enabled': True}, project='name', tag='computer')
    qb.append(Code, project='*', has_computer='computer', filters={
        'attributes.input_plugin': plugin_name,
        'or': [{'extras': {'!has_key': 'hidden'}}, {'extras.hidden': False}]
    })
    qb.order_by({Code: {'id': 'desc'}})
    codes = qb.all()
    sel_codes = []
    for code in codes:
        if code[0] == computer_name:
            sel_codes.append(code[1])
    return sel_codes

def on_computer_change(c):
    global cp2k_codes, eval_morb_codes, stm_image_codes
    cp2k_codes = comp_plugin_codes(drop_computer.value, 'cp2k')
    eval_morb_codes = comp_plugin_codes(drop_computer.value, 'spm.eval_morbs')
    stm_image_codes = comp_plugin_codes(drop_computer.value, 'spm.stm_image')
    
    drop_cp2k.options = [c.label for c in cp2k_codes]
    drop_eval.options = [c.label for c in eval_morb_codes]
    drop_imag.options = [c.label for c in stm_image_codes]
    
    
drop_computer.observe(on_computer_change)

drop_cp2k = ipw.Dropdown(description="Cp2k code")

drop_eval = ipw.Dropdown(description="Eval code")

drop_imag = ipw.Dropdown(description="Image code")

on_computer_change(0)

elpa_check = ipw.Checkbox(
    value=True,
    description='use ELPA',
    disabled=False
)

display(drop_computer, drop_cp2k, drop_eval, drop_imag, elpa_check)

# Scanning tunnelling microscopy (STM) parameters

Parameters for constant height (CH) and constant current (CC) STM pictures

In [ ]:
style = {'description_width': '140px'}
layout = {'width': '50%'}
layout_small = {'width': '25%'}

bias_voltages_text = ipw.Text(description='Voltages (V)',
                              value='-2.0 -1.5 -1.0 -0.5 -0.1 0.1 0.5 1.0 1.5 2.0',
                              style=style, layout=layout)

const_height_text = ipw.Text(description='Const. H (ang)',
                              value='3.0 5.0',
                              style=style, layout=layout)

const_current_text = ipw.Text(description='Const. cur. (isoval)',
                              value='1e-8 1e-7 1e-6',
                              style=style, layout=layout)

extrap_plane_floattext = ipw.BoundedFloatText(
                        description='Extrap plane (ang)',
                        min=1.0,
                        max=10.0,
                        step=0.1,
                        value=3.0,
                        style=style, layout=layout_small)

display(bias_voltages_text, const_height_text, const_current_text, extrap_plane_floattext)

Parameters for CH pictures of individual orbitals

In [ ]:
num_homo_inttext = ipw.IntText(
                    value=5,
                    description="num HOMO",
                    style=style, layout=layout_small)
num_lumo_inttext = ipw.IntText(
                    value=5,
                    description="num LUMO",
                    style=style, layout=layout_small)

display(num_homo_inttext, num_lumo_inttext)

Parameters for scanning tunneling spectroscopy (STS)

In [ ]:
sts_heights_text = ipw.Text(
                        description='STS heights (ang)',
                        value="3.0 5.0",
                        style=style, layout=layout)

#sts_isovalues_text = ipw.Text(
#                        description='STS isovalues',
#                        value="1e-8 1e-7",
#                        style=style, layout=layout)

sts_de_floattext = ipw.BoundedFloatText(
                        description='STS dE (eV)',
                        min=0.01,
                        max=0.10,
                        step=0.01,
                        value=0.05,
                        style=style, layout=layout_small)

sts_fwhm_floattext = ipw.BoundedFloatText(
                        description='STS FWHM (eV)',
                        min=0.05,
                        max=0.50,
                        step=0.01,
                        value=0.10,
                        style=style, layout=layout_small)

display(sts_heights_text, sts_de_floattext, sts_fwhm_floattext)

In [ ]:
def find_struct_wf(structure):
    qb = QueryBuilder()
    qb.append(Computer, filters={'enabled': True})
    selected_comp = qb.all()[drop_computer.index][0]
    
    # check stm
    extras = structure.get_extras()
    for ex_k in extras.keys():
        if ex_k.startswith(('stm', 'pdos', 'afm')):
            spm_workchain = load_node(extras[ex_k])
            cp2k_scf_calc = spm_workchain.get_outputs()[0]
            if cp2k_scf_calc.get_computer().hostname == selected_comp.hostname:
                wfn_path = cp2k_scf_calc.out.remote_folder.get_remote_path() + "/aiida-RESTART.wfn"
                # check if it exists
                file_exists = ! ssh {selected_comp.hostname} "if [ -f {wfn_path} ]; then echo 1 ; else echo 0 ; fi"
                if file_exists[0]:
                    print("Found .wfn from %s"%ex_k)
                    return wfn_path
    
    # check geo opt
    geo_opt_calc = structure.get_inputs()[0]
    geo_comp = geo_opt_calc.get_computer()
    if geo_comp.hostname == selected_comp.hostname:
        wfn_path = geo_opt_calc.out.remote_folder.get_remote_path() + "/aiida-RESTART.wfn"
        # check if it exists
        file_exists = ! ssh {selected_comp.hostname} "if [ -f {wfn_path} ]; then echo 1 ; else echo 0 ; fi"
        if file_exists[0]:
            print("Found .wfn from geo_opt")
            return wfn_path
    return ""

def on_submit(b):
    with submit_out:
        clear_output()
        if not struct_browser.results.value:
            print("Please select a structure.")
            return
        if not drop_computer.value:
            print("Please select a computer.")
            return
        
        bias_voltages = np.array(bias_voltages_text.value.split(), dtype=np.float)
        #height_values = np.array(const_height_text.value.split(), dtype=np.float)
        #current_values = np.array(const_current_text.value.split(), dtype=np.float)

        extrap_plane = extrap_plane_floattext.value
        #num_homo = int(num_homo_text.value)
        #num_lumo = int(num_lumo_text.value)

        #sts_heights = np.array(sts_heights_text.value.split(), dtype=np.float)
        #sts_de = float(sts_de_text.value)
        #sts_fwhm = float(sts_fwhm_text.value)
        
        parent_dir = "parent_calc/"

        eval_orbs_params = ParameterData(dict={
            '--cp2k_input':  parent_dir+'aiida.inp',
            '--basis_file':  parent_dir+'BASIS_MOLOPT',
            '--xyz_file':    parent_dir+'geom.xyz',
            '--wfn_file':    parent_dir+'aiida-RESTART.wfn',
            '--output_file': 'morbs_dx0.2',
            '--emin':        str(np.min(bias_voltages)-0.1),
            '--emax':        str(np.max(bias_voltages)+0.1),
            '--eval_region': ['G', 'G', 'G', 'G', 'b-1.5_C', 't%.1f'%extrap_plane],
            '--dx':          '0.2',
            '--eval_cutoff': '14.0'
        })

        stm_image_params = ParameterData(dict={
            '--npz_file':          parent_dir+'morbs_dx0.2.npz',
            '--hartree_file':      parent_dir+parent_dir+'aiida-HART-v_hartree-1_0.cube',
            '--extrap_plane':      str(extrap_plane),
            '--extrap_extent':     '5.0',
            '--output_dir':        'stm_output',
        #
            '--bias_voltages':     bias_voltages_text.value.split(),
            '--stm_plane_heights': const_height_text.value.split(),
            '--stm_isovalues':     const_current_text.value.split(),
        #
            '--sts_plane_heights': sts_heights_text.value.split(),
            '--sts_de':            str(sts_de_floattext.value),
            '--sts_fwhm':          str(sts_fwhm_floattext.value),
            '--sts_elim':          [str(np.min(bias_voltages)), str(np.max(bias_voltages))],
        #
            '--orb_plane_heights': const_height_text.value.split(),
            '--n_homo':            str(num_homo_inttext.value),
            '--n_lumo':            str(num_lumo_inttext.value),
            '--skip_figs':         ''
        })
        
        cp2k_code = cp2k_codes[drop_cp2k.index]
        eval_orbs_code = eval_morb_codes[drop_eval.index]
        stm_image_code = stm_image_codes[drop_imag.index]
        
        struct = struct_browser.results.value
        
        cell = ArrayData()
        cell.set_array('cell', np.diag(atoms.cell))
        
       ## Try to access the restart-wfn file ##
        try:
            wfn_file_path = find_struct_wf(struct)
        except:
            wfn_file_path = ""
        if wfn_file_path == "":
            print("Didn't find any accessible .wfn file.")
    
        
        outputs = submit(STMWorkChain,
                 cp2k_code=cp2k_code,
                 structure=struct,
                 cell=cell,
                 wfn_file_path=Str(wfn_file_path),
                 elpa_switch=Bool(elpa_check.value),
                 eval_orbs_code=eval_orbs_code,
                 eval_orbs_params=eval_orbs_params,
                 stm_image_code=stm_image_code,
                 stm_image_params=stm_image_params
                )
                
        print(outputs)

btn_submit = ipw.Button(description="Submit")
btn_submit.on_click(on_submit)
submit_out = ipw.Output()
display(btn_submit, submit_out)